In [1]:
#!pip install seaborn
#!pip install openpyxl


In [2]:
import pandas as pd
import numpy as np
import random as rd
import time
import csv
import seaborn as sbs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split ,GridSearchCV
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [3]:
#
# #from google.colab import drive
#drive.mount('/content/drive')
file_data = 'DS_Agua_2017_2022_por_ponto.csv'
path_name='data/'
path_name_results='results/'
file_result = 'Result_MLP_Model_Hour.csv'

In [4]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =';', encoding = 'latin1', decimal='.')


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 368849 entries, 1 to 368849
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   DT_MEDICAO_HORA         368849 non-null  object 
 1   PRECIPITACAO            368849 non-null  float64
 2   PRESSAO_ATMOSFERICA     368849 non-null  float64
 3   TEMPERATURA_DO_AR_C     368849 non-null  float64
 4   UMIDADE_RELATIVA_DO_AR  368849 non-null  float64
 5   VELOCIDADE_VENTO        368849 non-null  float64
 6   SK_PONTO                368849 non-null  int64  
 7   VL_MEDICAO              368849 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 25.3+ MB


In [6]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

DT_MEDICAO_HORA           0
PRECIPITACAO              0
PRESSAO_ATMOSFERICA       0
TEMPERATURA_DO_AR_C       0
UMIDADE_RELATIVA_DO_AR    0
VELOCIDADE_VENTO          0
SK_PONTO                  0
VL_MEDICAO                0
dtype: int64

In [7]:
def salvar_resultado(sk_ponto, ds_best_param, MSE, RMSE, MAE, MAPE, Duration):
  #Script to write training cycle results
  fields = ['SK_PONTO','Best Params','MSE','RMSE','MAE','MAPE','Duration']
  data = [sk_ponto, ds_best_param, MSE, RMSE, MAE, MAPE, Duration]
  with open(f'{path_name_results}{file_result}', "a") as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['SK_PONTO','Best Params','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w") as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)    

In [8]:
def previsao_MLP(sk_ponto, dataset):
  # dataframe tratament
  df = pd.DataFrame()
  df['VL_MEDICAO']=dataset['VL_MEDICAO']  

  # time serire transform - shit 1 step time
  seqtemp=1
  df['vl-1']=dataset['VL_MEDICAO'].shift(seqtemp)
  df['tp-1']=dataset['TEMPERATURA_DO_AR_C']
  df['pr-1']=dataset['PRESSAO_ATMOSFERICA']
  df['vv-1']=dataset['VELOCIDADE_VENTO']
  df['ur-1']=dataset['UMIDADE_RELATIVA_DO_AR']
  df['ch-1']=dataset['PRECIPITACAO']  

  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  75% treinam  25% test
  nlinhas = int(np.round(df.shape[0] *0.75)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1:7]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1:7] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  from sklearn.model_selection import TimeSeriesSplit
  # Cross-validated for time series
  ts_cv = TimeSeriesSplit(
      n_splits=5, # Number of divisions
      max_train_size=6785,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=2261, # maximum size of each set. of test.
  )
  param_grid = {
      'hidden_layer_sizes': [(4,6,1),(2,6,1),(6,12,1),(6,18,1)], # MLP layers
#    'hidden_layer_sizes': [(6,12,1),(6,12,24,1),(6,18,1)], # MLP layers
      'max_iter': [ 100, 150, 200], # maximum iterations
      'activation': [ 'relu','identity'], # activation function
      'solver': ['adam'], # weight optimization algorithm
      'alpha': [0.0001, 0.001, 0.05],  # alpha strength of regularization
  }
  
  from sklearn.neural_network import MLPRegressor
  modelo = MLPRegressor(random_state=0)
  
  grid = GridSearchCV(modelo, param_grid, n_jobs= -1,scoring='neg_mean_absolute_percentage_error', cv=ts_cv, verbose=1)
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))  

  # Stores the training execution end time
  Hora_Fim = time.time()  

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio  

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)  

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))  

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)  

  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100   

  salvar_resultado(sk_ponto, resultado, MSE, RMSE, MAE, MAPE, Duracao)

In [9]:
#selects the identification points of each reservoir
df_unique_pontos = dataset['SK_PONTO'].unique()

#create file to results
criar_arquivo_resultado()

for sk in np.array(df_unique_pontos):
    print('forecast for sk_ponto = ',sk)
    df_ponto = dataset[dataset['SK_PONTO']==sk]
    df_ponto = df_ponto.drop('SK_PONTO',axis=1)
    grid = previsao_MLP(sk, df_ponto)

forecast for sk_ponto =  5
Fitting 5 folds for each of 72 candidates, totalling 360 fits
['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[5, "{'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 200, 'solver': 'adam'}", 5929.058020502218, 77.00037675558619, 48.58768183965401, 6.90323067500058, 375.3172609806061]
forecast for sk_ponto =  6
Fitting 5 folds for each of 72 candidates, totalling 360 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[6, "{'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (4, 6, 1), 'max_iter': 100, 'solver': 'adam'}", 68684.89236394117, 262.078027243684, 250.7109329700351, 92.38962524388596, 254.2529513835907]
forecast for sk_ponto =  7
Fitting 5 folds for each of 72 candidates, totalling 360 fits
['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[7, "{'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (4, 6, 1), 'max_iter': 150, 'solver': 'adam'}", 1929.5134627360912, 43.9262275040333, 24.31149361511953, 5.4027631013838295, 457.8954265117645]
forecast for sk_ponto =  11
Fitting 5 folds for each of 72 candidates, totalling 360 fits
